In [1]:
import pandas as pd
import re
from pprint import pprint

# Reading Code Book
[The Culper Code Book](http://www.mountvernon.org/george-washington/the-revolutionary-war/spying-and-espionage/the-culper-code-book/)

[Cryptology in the American Revolution](https://www.nsa.gov/about/cryptologic-heritage/historical-figures-publications/publications/pre-wwii/assets/files/Revolutionary_Secrets_2012.pdf)

In [2]:
dfCode = pd.read_excel('./RevolutionaryWarKey.xlsx')
dfCode = dfCode.loc[dfCode['isDrop']!=True]
dfCode['Code'] = dfCode['Code'].astype(str)
dfCode['PlainText'] = dfCode['PlainText'].astype(str)
dfCode['CodeConv'] = dfCode['Code'].str.strip()
dfCode['PlainTextConv'] = dfCode['PlainText'].str.strip()
dfCode['CodeConv'] = dfCode['CodeConv'].str.lower()
dfCode['PlainTextConv'] = dfCode['PlainTextConv'].str.lower()
print(dfCode.shape)
dfCode.head(10)

(798, 6)


,Code,PlainText,Category,isDrop,CodeConv,PlainTextConv
0,A,e,Alphabet,NaN,a,e
1,B,f,Alphabet,NaN,b,f
2,C,g,Alphabet,NaN,c,g
3,D,h,Alphabet,NaN,d,h
4,E,i,Alphabet,NaN,e,i
5,F,j,Alphabet,NaN,f,j
6,G,a,Alphabet,NaN,g,a
7,H,b,Alphabet,NaN,h,b
8,I,c,Alphabet,NaN,i,c
9,J,d,Alphabet,NaN,j,d


# Functions for Enciphering & Deciphering

In [3]:
def getCulperEncipheredText(strVal):
    rtnVal = strVal
    strVal = strVal.strip().lower()
    tmp = dfCode.loc[dfCode['PlainTextConv']==strVal]
    if(tmp.shape[0]>0):
        if(tmp.shape[0]>1):
            rtnVal = '('+'|'.join(tmp['Code'].tolist())+')'
        else:
            rtnVal = tmp.iloc[0]['Code']
    elif(len(strVal)>1):
        tmpLetters = list(strVal)
        rtnVal = ''.join(list(map(getCulperEncipheredText,tmpLetters)))
    return(rtnVal)

In [4]:
def getCulperDecipheredText(strVal):
    rtnVal = strVal
    strVal = strVal.strip().lower()
    tmp = dfCode.loc[dfCode['CodeConv']==strVal]
    if(tmp.shape[0]>0):
        if(tmp.shape[0]>1):
            rtnVal = '('+'|'.join(tmp['PlainText'].tolist())+')'
        else:
            rtnVal = tmp.iloc[0]['PlainText']
    elif(len(strVal)>1):
        tmpLetters = list(strVal)
        rtnVal = ''.join(list(map(getCulperDecipheredText,tmpLetters)))
    return(rtnVal)

In [5]:
def getEncryptedText(strVal):
    valMsg01 = strTokenize(strVal)
    valMsg02 = list(map(getCulperEncipheredText, valMsg01))
    rtnVal = ' '.join(valMsg02).replace(' .','.')
    return(rtnVal)

In [6]:
def getDecryptedText(strVal):
    valMsg01 = strTokenize(strVal)
    valMsg02 = list(map(getCulperDecipheredText, valMsg01))
    rtnVal = ' '.join(valMsg02).replace(' .','.')
    return(rtnVal)

## Testing
### Enciphering Text

In [7]:
getCulperEncipheredText('e')

'A'

In [8]:
getCulperEncipheredText('E')

'A'

In [9]:
getCulperEncipheredText('MaRYlaND')

'738'

In [10]:
getCulperEncipheredText('Longfellow')

'RKMCBARRKU'

In [11]:
getCulperEncipheredText("5")

'k'

### Deciphering

In [12]:
getCulperDecipheredText('a')

'e'

In [13]:
getCulperDecipheredText('A')

'e'

In [14]:
getCulperDecipheredText('738')

'Maryland'

In [15]:
getCulperDecipheredText('RKMCBARRKU')

'l(o|5)(n|6)gfell(o|5)(w|0)'

In [16]:
getCulperDecipheredText('k')

'(o|5)'

# Functions for Messages

In [17]:
def strTokenize(strVal):
    rtnVal = re.findall(r"[\w']+|[.,!?;-]", strVal)
    return(rtnVal)

## Testing

In [18]:
%%time
valMsg01 = strTokenize("This is a test of the string tokenizer.  Does it work?")
pprint(valMsg01)
valMsg02 = list(map(getCulperEncipheredText, valMsg01))
pprint(valMsg02)
valMsg03 = list(map(getCulperDecipheredText, valMsg02))
pprint(valMsg03)

['This',
 'is',
 'a',
 'test',
 'of',
 'the',
 'string',
 'tokenizer',
 '.',
 'Does',
 'it',
 'work',
 '?']
['627',
 '283',
 '(G|1)',
 'ZAYZ',
 '431',
 '625',
 'YZPEMC',
 'ZKQAMEXAP',
 '.',
 'JKAY',
 '284',
 'UKPQ',
 '617']
['this',
 'is',
 '((a|3)|a)',
 'test',
 'of',
 'the',
 'str(i|1)(n|6)g',
 't(o|5)(k|9)e(n|6)(i|1)zer',
 '.',
 'd(o|5)es',
 'it',
 '(w|0)(o|5)r(k|9)',
 '?']
CPU times: user 86.2 ms, sys: 17.3 ms, total: 104 ms
Wall time: 89.1 ms


In [19]:
pprint(list(map(getCulperEncipheredText, strTokenize("1 if by land, 2 if by sea - Longfellow"))))

['e', '281', '50', '347', ',', 'f', '281', '50', '588', '-', 'RKMCBARRKU']


In [20]:
tmpVal = list("Longfellow")
''.join(list(map(getCulperEncipheredText, tmpVal)))

'RKMCBARRKU'

# Examples
## Using the Tallmadge Cipher Simplistically

In [21]:
%%time
strTallmadgePlain00 = "There is no excuse for famine in Boston, please send ammunition."
strTallmadgeEncoded00 = ' '.join(list(map(getCulperEncipheredText, strTokenize(strTallmadgePlain00)))).replace(' .','.')
print(strTallmadgeEncoded00)

630 283 413 169 BKP 194 282 733 , 475 591 47.
CPU times: user 19.1 ms, sys: 2.53 ms, total: 21.7 ms
Wall time: 19.6 ms


## Real Example

In [22]:
%%time
strTallmadgePlain01 = '''Dr Sir
I have received Your Letter of the 6th Inst.
The success of the enterprise proposed, must depend on the absence of the British Fleet, the secrecy of the Attempt, and a knowledge of the exact situation of the Enemy. If after you have been at the Westward, the circumstances, from your intelligence, shall appear favorable; You will be at liberty, to be the bearer of the inclosed Letter to His Excellency the Count De Rochambeau, to whose determination, I have referred the Matter; as any cooperation on our part by Moving Troops towards the Sound, would give such indications of the design as would effectually frustrate the success.
Should you not proceed to the Count you may destroy that Letter—If on the contrary you should go to New Port, by keeping an account of the expences, they will be repaid by the Public.
In the mean time, I wish you to be as particular as possible, in obtaining from your friend, an accurate account of the Enemy’s strength, on York, Long, and Staten Islands, specifying the several Corps, and their distributions. This I think from the Enemy’s present weak state, may be procured with more facility & accuracy, than at any former Period. I am Sir Your Most Obedient Servant
Go: Washington
P.S. I wish to know also, the strength of the last Detachment from New York, and of what Troops it was composed.
I need scarcely suggest, if you should go Eastward that it will be expedient to do it in such a manner as not to create suspicion—indeed, you know, secrecy is absolutely necessary in the whole affair.
As the Count De Rochambeau does not understand English, it may be well to communicate Your business to the Chevalier De Chattelus in the first instance, and thro’ him to the Count, lest it should by accident get abroad.
'''
strTallmadgeEncoded01 = ' '.join(list(map(getCulperEncipheredText, strTokenize(strTallmadgePlain01)))).replace(' .','.')
print(strTallmadgeEncoded01)

JP YEP E 249 PAIAETAJ 708 356 431 625 mZD EMYZ. 625 YSIIAYY 431 625 AMZAPNPEYA NPKNKYAJ , LSYZ JANAMJ 433 625 (G|1)HYAMIA 431 625 72 193 , 625 YAIPAIW 431 625 36 , 5 (G|1) QMKURAJCA 431 625 AV(G|1)IZ YEZS(G|1)ZEKM 431 625 178. 281 (G|1)BZAP 707 249 HAAM 4 625 UAYZU(G|1)PJ , 625 IEPISLYZ(G|1)MIAY , BPKL 708 EMZARRECAMIA , YD(G|1)RR 27 B(G|1)TKP(G|1)HRA ; 707 683 48 4 368 , 634 48 625 HA(G|1)PAP 431 625 EMIRKYAJ 356 634 243 AVIARRAMIW 625 IKSMZ JA PKID(G|1)LHA(G|1)S , 634 694 JAZAPLEM(G|1)ZEKM , E 249 PABAPPAJ 625 L(G|1)ZZAP ; (G|1)Y 26 IKKNAP(G|1)ZEKM 433 KSP 476 50 LKTEMC 635 ZKU(G|1)PJY 625 YKSMJ , UKSRJ CETA YSID EMJEI(G|1)ZEKMY 431 625 JAYECM (G|1)Y UKSRJ ABBAIZS(G|1)RRW BPSYZP(G|1)ZA 625 YSIIAYY. YDKSRJ 707 414 NPKIAAJ 634 625 IKSMZ 707 373 JAYZPKW 626 356 281 433 625 IKMZP(G|1)PW 707 YDKSRJ 220 634 412 473 , 50 QAANEMC 2 (G|1)IIKSMZ 431 625 AVNAMIAY , 629 683 48 PAN(G|1)EJ 50 625 NSHREI. 282 625 LA(G|1)M 633 , E 693 707 634 48 (G|1)Y 526 (G|1)Y NKYYEHRA , 282 KHZ(G|1)EMEMC BPKL 70

In [23]:
%%time
strTallmadgeDecoded01 = ' '.join(list(map(getCulperDecipheredText, strTokenize(strTallmadgeEncoded01)))).replace(' .','.')
print(strTallmadgeDecoded01)

dr s(i|1)r (i|1) have re(c|4)e(i|1)ved your letter of the (n|6)th (i|1)(n|6)st. the su(c|4)(c|4)ess of the e(n|6)ter(p|7)r(i|1)se (p|7)r(o|5)(p|7)(o|5)sed , must de(p|7)e(n|6)d on the (a|3) a bse(n|6)(c|4)e of the british fleet , the se(c|4)re(c|4)y of the attempt , and (a|3) a (k|9)(n|6)(o|5)(w|0)ledge of the ex (a|3) a (c|4)t s(i|1)tu (a|3) a t(i|1)(o|5)(n|6) of the enemy. if (a|3) a fter you have bee(n|6) at the (w|0)est(w|0) (a|3) a rd , the (c|4)(i|1)r(c|4)umst (a|3) a (n|6)(c|4)es , fr(o|5)m your (i|1)(n|6)tell(i|1)ge(n|6)(c|4)e , sh (a|3) a ll appear f (a|3) a v(o|5)r (a|3) a ble ; you will be at liberty , to be the be (a|3) a rer of the (i|1)(n|6)(c|4)l(o|5)sed letter to his ex(c|4)elle(n|6)(c|4)y the (c|4)(o|5)u(n|6)t de r(o|5)(c|4)h (a|3) a mbe (a|3) a u , to whose determ(i|1)(n|6) (a|3) a t(i|1)(o|5)(n|6) , (i|1) have referred the m (a|3) a tter ; (a|3) a s any (c|4)(o|5)(o|5)(p|7)er (a|3) a t(i|1)(o|5)(n|6) on (o|5)ur part by m(o|5)v(i|1)(n|6)g troops t(o|5)(w|0) (a|3) a rd

In [24]:
getEncryptedText(strTallmadgePlain01)

'JP YEP E 249 PAIAETAJ 708 356 431 625 mZD EMYZ. 625 YSIIAYY 431 625 AMZAPNPEYA NPKNKYAJ , LSYZ JANAMJ 433 625 (G|1)HYAMIA 431 625 72 193 , 625 YAIPAIW 431 625 36 , 5 (G|1) QMKURAJCA 431 625 AV(G|1)IZ YEZS(G|1)ZEKM 431 625 178. 281 (G|1)BZAP 707 249 HAAM 4 625 UAYZU(G|1)PJ , 625 IEPISLYZ(G|1)MIAY , BPKL 708 EMZARRECAMIA , YD(G|1)RR 27 B(G|1)TKP(G|1)HRA ; 707 683 48 4 368 , 634 48 625 HA(G|1)PAP 431 625 EMIRKYAJ 356 634 243 AVIARRAMIW 625 IKSMZ JA PKID(G|1)LHA(G|1)S , 634 694 JAZAPLEM(G|1)ZEKM , E 249 PABAPPAJ 625 L(G|1)ZZAP ; (G|1)Y 26 IKKNAP(G|1)ZEKM 433 KSP 476 50 LKTEMC 635 ZKU(G|1)PJY 625 YKSMJ , UKSRJ CETA YSID EMJEI(G|1)ZEKMY 431 625 JAYECM (G|1)Y UKSRJ ABBAIZS(G|1)RRW BPSYZP(G|1)ZA 625 YSIIAYY. YDKSRJ 707 414 NPKIAAJ 634 625 IKSMZ 707 373 JAYZPKW 626 356 281 433 625 IKMZP(G|1)PW 707 YDKSRJ 220 634 412 473 , 50 QAANEMC 2 (G|1)IIKSMZ 431 625 AVNAMIAY , 629 683 48 PAN(G|1)EJ 50 625 NSHREI. 282 625 LA(G|1)M 633 , E 693 707 634 48 (G|1)Y 526 (G|1)Y NKYYEHRA , 282 KHZ(G|1)EMEMC BPKL 7

In [25]:
getDecryptedText(getEncryptedText(strTallmadgePlain01))

'dr s(i|1)r (i|1) have re(c|4)e(i|1)ved your letter of the (n|6)th (i|1)(n|6)st. the su(c|4)(c|4)ess of the e(n|6)ter(p|7)r(i|1)se (p|7)r(o|5)(p|7)(o|5)sed , must de(p|7)e(n|6)d on the (a|3) a bse(n|6)(c|4)e of the british fleet , the se(c|4)re(c|4)y of the attempt , and (a|3) a (k|9)(n|6)(o|5)(w|0)ledge of the ex (a|3) a (c|4)t s(i|1)tu (a|3) a t(i|1)(o|5)(n|6) of the enemy. if (a|3) a fter you have bee(n|6) at the (w|0)est(w|0) (a|3) a rd , the (c|4)(i|1)r(c|4)umst (a|3) a (n|6)(c|4)es , fr(o|5)m your (i|1)(n|6)tell(i|1)ge(n|6)(c|4)e , sh (a|3) a ll appear f (a|3) a v(o|5)r (a|3) a ble ; you will be at liberty , to be the be (a|3) a rer of the (i|1)(n|6)(c|4)l(o|5)sed letter to his ex(c|4)elle(n|6)(c|4)y the (c|4)(o|5)u(n|6)t de r(o|5)(c|4)h (a|3) a mbe (a|3) a u , to whose determ(i|1)(n|6) (a|3) a t(i|1)(o|5)(n|6) , (i|1) have referred the m (a|3) a tter ; (a|3) a s any (c|4)(o|5)(o|5)(p|7)er (a|3) a t(i|1)(o|5)(n|6) on (o|5)ur part by m(o|5)v(i|1)(n|6)g troops t(o|5)(w|0) (a|3) a r